## Lab 7: Compression

In [50]:
# Imports
import numpy as np

### Simulating the data

In [49]:
# Generate 100 MB of random binary data containing 100%, 90%, 80%, 70%, 60%, and 50% zeros
percentages = np.arange(50, 110, 10)

for percentage in percentages:
    rand_data = np.random.choice([0, 1], size=8*1024*1024*100, replace=True, p=[percentage/100,(1-(percentage/100))])
    rand_data = np.packbits(rand_data)
    open('zeros_'+str(percentage)+'p', 'wb').write(rand_data)

In [52]:
# Generate DNA and protein sequences 100 million letters long, with each letter of equal probability
seq_size = 100000000
nt_letters = ['A','C','G','T']

nt_seq = np.random.choice(nt_letters, size=seq_size, replace=True)
open('nt_seq.fa', 'w').write(''.join(nt_seq))

100000000

[Amino acid alphabet](http://www.afrimage.org/amino-acid-alphabet/)

In [53]:
prot_letters = ['A','R','N','D','C','Q',',E','G','H','I','L','K','M','F','P','S','T','W','Y','V']

prot_seq = np.random.choice(prot_letters, size=seq_size, replace=True)
open('prot_seq.fa', 'w').write(''.join(prot_seq))

105003247

### Compressing the data

#### Terminal commands (repeated for all uncompressed input files)
    be131-12@meowth:~/jess/Lab7$ time gzip -k zeros_100p

    real    0m0.733s
    user    0m0.692s
    sys     0m0.040s
    be131-12@meowth:~/jess/Lab7$ time bzip2 -k zeros_100p

    real    0m1.054s
    user    0m0.986s
    sys     0m0.069s
    be131-12@meowth:~/jess/Lab7$ cp zeros_100p zeros_100p_pbzip
be131-12@meowth:~/jess/Lab7$ time pbzip2 -k zeros_100p_pbzip

    real    0m0.107s
    user    0m1.968s
    sys     0m0.120s
    be131-12@meowth:~/jess/Lab7$ time ArithmeticCompress zeros_100p zeros_100p.art

    real    0m14.956s
    user    0m14.744s
    sys     0m0.213s

#### Time and Memory of 4 Different Compression Methods on Simulated Data

|Sequence|Input file size|Compression method|Output file size|Runtime (real)|
| ------ |:-------------:|-----------------:|---------------:|:------------:|
|100% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|102 kB<br>113 B<br>5.62 kB<br>1.03 kB|0.733s<br>0.962s<br>0.005s<br>14.956s|
|90% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|58.7 MB<br>61.2 MB<br>61.2 MB<br>49.2 MB|18.864s<br>10.646s<br>0.768s<br>28.939s|
|80% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|81.2 MB<br>86.6 MB<br>86.7 MB<br>75.7 MB|13.361s<br>12.009s<br>0.938s<br>35.362s|
|70% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|93.6 MB<br>99.8 MB<br>99.8 MB<br>92.4 MB|6.061s<br>13.856s<br>1.176s<br>39.249s|
|60% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|102 MB<br>105 MB<br>105 MB<br>102 MB|4.291s<br>15.722s<br>1.287s<br>41.070s|
|50% zeros|105 MB|gz<br>bz2<br>pbz2<br>art<br>|105 MB<br>105 MB<br>105 MB<br>105 MB|3.562s<br>16.668s<br>1.537s<br>41.434s|
|Nucleotide|100 MB|gz<br>bz2<br>pbz2<br>art<br>|29.2 MB<br>27.3 MB<br>27.3 MB<br>25 MB|12.151s<br>9.472s<br>0.680s<br>21.319s|
|Protein|105 MB|gz<br>bz2<br>pbz2<br>art<br>|61.9 MB<br>55.3 MB<br>55.3 MB<br>57.7 MB|5.965s<br>10.468s<br>0.800s<br>30.236s|

**Which algorithm achieves the best level of compression on each file type?**  
Binary files  
100%: bzip  
90% + 80%: gzip  
70%:  ArithmeticCompress  
60%: tie between gzip and ArithmeticCompress  
50%: tie between all algorithms  

Sequence files  
nucleotide: ArithmeticCompress  
protein: tie between bzip and pbzip2  

**Which algorithm is the fastest?**  
pbzip2 was the fastest algorithm for all files.  

**What is the difference between bzip2 and pbzip2? Do you expect one to be faster and why?**  
According to the Linux manual page for pbzip:  
>"pbzip2 is a parallel implementation of the bzip2 block-sorting file compressor that uses pthreads and achieves near-linear speedup on SMP machines. The output of this version is fully compatible with bzip2 v1.0.2 or newer (ie: anything compressed with pbzip2 can be decompressed with bzip2).  
>pbzip2 should work on any system that has a pthreads compatible C++ compiler (such as gcc). It has been tested on: Linux, Windows (cygwin), Solaris, Tru64/OSF1, HP-UX, and Irix."  

So because pbzip2 uses essentially the same algorithm as bzip2 compression but uses multiple threads to implement bzip2 compression in parallel, it makes sense for pbzip2 to be faster.  

**How does the level of compression change as the percentage of zeros increases? Why does this happen?**  
Compression ratio = Uncompressed size/Compressed size  

|% zeros|Input file size|Compression method|Output file size|Compression ratio|Mean comp. ratio|
| ----- |:-------------:|-----------------:|---------------:|:---------------:|------------------------:|
|100%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|102 kB<br>113 B<br>5.62 kB<br>1.03 kB|1054<br>9.74 x 10<sup>5</sup><br>1.91 x 10<sup>4</sup><br>1.04 x 10<sup>5</sup>|2.75 x 10<sup>5</sup>|
|90%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|58.7 MB<br>61.2 MB<br>61.2 MB<br>49.2 MB|1.79<br>1.72<br>1.72<br>2.13|1.84|
|80%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|81.2 MB<br>86.6 MB<br>86.7 MB<br>75.7 MB|1.29<br>1.21<br>1.21<br>1.39|1.27|
|70%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|93.6 MB<br>99.8 MB<br>99.8 MB<br>92.4 MB|1.12<br>1.05<br>1.05<br>1.14|1.09|
|60%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|102 MB<br>105 MB<br>105 MB<br>102 MB|1.03<br>1.00<br>1.00<br>1.03|1.01|
|50%|105 MB|gz<br>bz2<br>pbz2<br>art<br>|105 MB<br>105 MB<br>105 MB<br>105 MB|1.00<br>1.00<br>1.00<br>1.00|1.00|  

As the percentage of zeros increases, the level of compression increases. This is because files constructed from probability distributions with increasing percentages of zeros have a smaller expected Shannon information (entropy) needed to encode any outcome from the probability distribution. This means each position in a file contains less Shannon information on average as the percentage of zeros increases, so that an "ideal" data compression algorithm would perform better on it (greater compression ratio.)  

**What is the minimum number of bits required to store a single DNA base?**  
S = log<sub>2</sub>4 = 2 bits  

**What is the minimum number of bits required to store an amino acid letter?**  
S = log<sub>2</sub>20 = 4.32 bits  

**In your tests, how many bits did gzip and bzip2 actually require to store your random DNA and protein sequences?**  

|sequence| gzip  | bzip2 |
|------- |------:|------:|
|  DNA   |29.2 MB|27.3 MB|
|protein |61.9 MB|55.3 MB|

Min bits in DNA sequence ("ideal") = 10 x 10<sup>6</sup> positions x 2 bits/position = 2.0 x 10<sup>8</sup> bits  
Actual gzip performance = 29.2 MB x 1024 kB/MB x 1024 B/kB x 8 bits/byte = 2.45 x 10<sup>8</sup> bits
Actual bzip2 performance = 27.3 MB x 1024 kB/MB x 1024 B/kB x 8 bits/byte = 2.20 x 10<sup>8</sup> bits  

Min bits in protein sequence ("ideal") = 10 x 10<sup>6</sup> positions x 4.32 bits/position = 4.32 x 10<sup>8</sup> bits  
Actual gzip performance = 61.9 MB x 1024 kB/MB x 1024 B/kB x 8 bits/byte = 5.19 x 10<sup>8</sup> bits
Actual bzip2 performance = 55.3 MB x 1024 kB/MB x 1024 B/kB x 8 bits/byte = 4.64 x 10<sup>8</sup> bits  

**Are gzip and bzip2 performing well on DNA and proteins?**  
Both gzip and bzip2 perform fairly well on DNA and protein sequences because they are able to encode the information in the sequences using an amount of bits fairly close to the minimum bits needed to encode those sequences (achieved by an "ideal" data compression algorithm), as shown in the calculations above. In both the DNA and protein sequences, bzip2 achieved data compression closer to "ideal" compression compared to gzip.  

### Compressing real data
Using what you’ve learned about querying biological databases, find the
nucleic acid sequences of gp120 homologs from at least 10 different HIV isolates and concatenate them together into a single multi-FASTA.  

**A priori, do you expect to achieve better or worse compression here than random data? Why?**  
I expect to achieve worse compression than on the simulated random data. This is because the FASTA format contains extra information in the form of the headers for each sequence. In addition, I expect the nucleic acid sequences to be fairly uniformly randomly distributed, similar to in the simulated nucleotide sequences.  

In [77]:
# Find the nucleic acid sequences of gp120 homologs from at least 10 different HIV isolates and
# concatenate them together into a single multi-FASTA.

from Bio import Entrez, SeqIO
# Define Entrez arguments
Entrez.email = 'jessica.wu@berkeley.edu'
query = 'gp120'

fa = open('gp120.fa', 'w')
num_homologs = 10
for n in range(num_homologs):
    # Get sequence field from nucleotide database 
    nt_handle = Entrez.esearch(db='nucleotide', term=query, sort='relevance')
    i = Entrez.read(nt_handle)['IdList'][n] 
    fasta_handle = Entrez.efetch(db='nucleotide', id=i, rettype='fasta', retmode='text')

    # Parse FASTA
    for record in SeqIO.parse(fasta_handle, "fasta"):
        fa.write('>' + record.description)
        fa.write(str(record.seq) + '\n')

fa.close()

#### Compress the multi-FASTA using gzip, bzip2, and arithmetic coding
#### Terminal commands:  
    be131-12@meowth:~/jess/Lab7$ time gzip -k gp120.fa
    
    real    0m0.003s
    user    0m0.000s
    sys     0m0.003s
    be131-12@meowth:~/jess/Lab7$ time bzip2 -k gp120.fa

    real    0m0.006s
    user    0m0.006s
    sys     0m0.000s
    be131-12@meowth:~/jess/Lab7$ cp gp120.fa gp120_pbzip2.fa
be131-12@meowth:~/jess/Lab7$ time pbzip2 -k gp120_pbzip2.fa

    real    0m0.008s
    user    0m0.004s
    sys     0m0.004s
    be131-12@meowth:~/jess/Lab7$ time ArithmeticCompress gp120.fa gp120.fa.art

    real    0m0.011s
    user    0m0.010s
    sys     0m0.001s


#### Time and Memory of 4 Different Compression Methods on Real Data (HIV gp120 multi-FASTA)  
Input file size: 7.24 kB

|Compression Method|Output file size|Runtime (real)|
|-----------------:|---------------:|:------------:|
|       gzip       |     1.9 kB     |    0.003s    |
|       bzip2      |     1.97 kB    |    0.006s    |
|      pbzip2      |     1.97 kB    |    0.008s    |
|ArithmeticCompress|     3.45 kB    |    0.011s    |

**How does the compression ratio of this file compare to random data?**  
Compression ratio = Uncompressed size/Compressed size  

|  DNA Data  |Input file size|Compression method|Output file size|Compression ratio|Mean comp. ratio|
| ---------- |-----------------:|---------------:|:---------------:|------------------------:|
|   Random   |100 MB|gz<br>bz2<br>pbz2<br>art<br>|29.2 MB<br>27.3 MB<br>27.3 MB<br>25 MB|3.42<br>3.66<br>3.66<br>4.00|3.69|
|    Real    |7.24 kB|gz<br>bz2<br>pbz2<br>art<br>|1.9 kB<br>1.97 kB<br>1.97 kB<br>3.45 kB|3.81<br>3.68<br>3.68<br>2.10|3.32|

Contrary to what I hypothesized, the file with real data had a slightly better compression ratio than of the simulated random data. I think this is because the the nucleic acid sequences are not as uniformly randomly distributed as I thought they would be. In order for the real sequences to have better compression, they have to have a lower expected Shannon information content (entropy) than the simulated random nucleotide sequence.  

### Estimating compression of 1000 terabytes

Let’s make some assumptions about the contents of the data at your biotech company. Most of the data, say 80%, is re-sequencing of genomes and plasmids that are very similar to each other. Another 10% might be protein sequences, and the last 10% are binary microscope images which we’ll assume follow the worst-case scenario of being completely random. 

Protein sequence compression performance:

|Input file size|Compression method|Output file size|Compression ratio|Mean comp. ratio|
| ------------- |-----------------:|---------------:|:---------------:|---------------:|
|105 MB|gz<br>bz2<br>pbz2<br>art<br>|61.9 MB<br>55.3 MB<br>55.3 MB<br>57.7 MB|1.70<br>1.90<br>1.90<br>1.82|1.83|

**Given the benchmarking data you obtained in this lab, which algorithm do you propose to use for each type of data? Provide an estimate for the fraction of space you can save using your compression scheme. How much of a bonus do you anticipate receiving this year?**  

|Data type|Compression method|Compression ratio|Mean comp. ratio|
| ------- |-----------------:|----------------:|---------------:|
|Real DNA sequences|gz<br>bz2<br>pbz2<br>art<br>|3.81<br>3.68<br>3.68<br>2.10|3.32|
|Protein sequences|gz<br>bz2<br>pbz2<br>art<br>|1.70<br>1.90<br>1.90<br>1.82|1.83|
|Random binary|gz<br>bz2<br>pbz2<br>art<br>|1.00<br>1.00<br>1.00<br>1.00|1.00|  

According to the benchmarking data, I would use gzip for the real DNA sequencing data and pbzip2 for the protein sequences (same compression ratio but faster than bzip2.) I wouldn't bother compressing the completely random binary data because essentially no compression is possible using any of the algorithms, and given the large amount of data (10% of 1000 TB = 100 TB), it would be a waste of time.

#### Estimate
weighted compression ratio = 0.8(3.81) + 0.1(1.90) + 0.1(1) = 3.34  
compressed data = 1000 TB/3.34 = 299.4 TB  
(1000 TB - 299.4 TB)/1000 TB x 100 = 70% data reduction  
70% reduction x (\$500/day/1% reduction) x (365 days/year) = $12.8 million bonus